In [1]:
from custom_envs.dm_control_suite import get_default_config, load, get_domain_randomizer
# from custom_envs.manipulation import get_default_config, load, get_domain_randomizer
import jax
import jax.numpy as jnp
import os
from learning.module.wrapper.adv_wrapper import wrap_for_adv_training
from learning.module.wrapper.dr_wrapper import wrap_for_dr_training
import functools
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]='false'
import mediapy as media


In [2]:
env_name='CartpoleSwingup'
cfg = get_default_config(env_name)
env = load(env_name, cfg)
# randomizer = get_domain_randomizer(env_name)
# randomizer = functools.partial(randomizer, dr_range=env.dr_range)

# eval_env = wrap_for_adv_training(
#       env,
#       episode_length=1000,
#       action_repeat=2,
#       randomization_fn=randomizer,
# )
# eval_env2 = wrap_for_dr_training(
#       env,
#       episode_length=1000,
#       action_repeat=2,
#       randomization_fn=randomizer,
#       n_envs=1,
#       n_nominals=1,
# )


In [3]:
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)


In [4]:
init_state = jit_reset(jax.random.PRNGKey(2))
state = init_state


In [5]:
seconds = 3
rollout=[state]
for i in range(int(seconds/env.sim_dt)):
    state = jit_step(state, jnp.ones(env.action_size))
    rollout.append(state)
frames = env.render(rollout)
media.show_video(frames, fps=1.0 / env.sim_dt)


100%|██████████| 301/301 [00:00<00:00, 313.41it/s]


This browser does not support the video tag.


In [6]:
state.data.xpos


Array([[0.       , 0.       , 0.       ],
       [1.8029578, 0.       , 1.       ],
       [1.8029578, 0.       , 1.       ]], dtype=float32)

In [7]:
from learning.module.wrapper.evaluator import AdvEvaluator


evaluator = AdvEvaluator(
        env,
        lambda x : x,
        num_eval_envs=128,
        episode_length=1000,
        action_repeat=2,
        key=jax.random.PRNGKey(0),
    )


In [ ]:
dynamics_params = jnp.zeros((128,14))
metrics = evaluator.run_evaluation(dynamics_params, dynamics_params, {})


ValueError: split accepts a single key, but was given a key array of shape (128, 2) != (). Use jax.vmap for batching.

: 

In [ ]:
env.mjx_model.body_ipos


In [ ]:
state.data.geom_xpos


In [ ]:
env.mjx_model.geom_friction


In [ ]:
env.mjx_model.dof_frictionloss.shape


(13,)

In [ ]:
env.mjx_model.nv


13

In [ ]:
env.mjx_model.nu


5

In [ ]:
env.mjx_model.qpos0.shape


In [ ]:
from mujoco import mj_id2name, mj_name2id
import mujoco
[mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i) for i in range(env.mj_model.nbody)]


['world',
 'head',
 'segment_0',
 'segment_1',
 'segment_2',
 'segment_3',
 'segment_4',
 'target']

In [ ]:

[mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_JOINT, i) for i in range(env.mj_model.njnt)]



['rootx',
 'rooty',
 'rootz',
 'joint_0',
 'joint_1',
 'joint_2',
 'joint_3',
 'joint_4']

In [ ]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_GEOM, i), 
        env.mj_model.geom_friction[i]
    ) 
    for i in range(env.mj_model.ngeom)
]


[('ground', array([1.e+00, 5.e-03, 1.e-04])),
 ('head', array([1.e+00, 5.e-03, 1.e-04])),
 ('nose', array([1.e+00, 5.e-03, 1.e-04])),
 ('eyes', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual_0', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial_0', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual_1', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial_1', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual_2', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial_2', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual_3', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial_3', array([1.e+00, 5.e-03, 1.e-04])),
 ('visual_4', array([1.e+00, 5.e-03, 1.e-04])),
 ('inertial_4', array([1.e+00, 5.e-03, 1.e-04])),
 ('target', array([1.e+00, 5.e-03, 1.e-04]))]

In [ ]:
env.mj_model.geom_friction[0]



array([1.e+00, 5.e-03, 1.e-04])

In [ ]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i), 
        env.mj_model.body_mass[i]
    ) 
    for i in range(env.mj_model.nbody)
]


[('world', np.float64(0.0)),
 ('head', np.float64(0.01)),
 ('segment_0', np.float64(0.01)),
 ('segment_1', np.float64(0.01)),
 ('segment_2', np.float64(0.01)),
 ('segment_3', np.float64(0.01)),
 ('segment_4', np.float64(0.01)),
 ('target', np.float64(4.188790204786391))]

In [ ]:

init_state=  env.reset(jax.random.PRNGKey(0))
state=init_state



In [ ]:
from custom_envs import mjx_env


state.data
mjx_env.get_sensor_data(
        env.mj_model, state.data, "torso_subtreelinvel"
    )


Array([0., 0., 0.], dtype=float32)

In [ ]:
state = env.step(state, jnp.ones(env.action_size))
state.data.xpos



Array([[ 0.        ,  0.        ,  0.        ],
       [ 0.00229061, -0.09994751,  0.05      ],
       [ 0.04339438, -0.18141547,  0.05      ],
       [ 0.13138694, -0.20391439,  0.05      ],
       [ 0.22999373, -0.20243815,  0.05      ],
       [ 0.3164212 , -0.24361372,  0.05      ],
       [ 0.3989326 , -0.29794952,  0.05      ],
       [-1.3853827 ,  0.91602135,  0.05      ]], dtype=float32)

In [ ]:
import jax.numpy as jnp
a = jnp.arange(10)


In [ ]:
a[2:10:3]
